# 概要

機械学習において重要トピックの一つであるハイパーパラメータのチューニング方法を実装してみる。

今回california_housingをつかって、ランダムサーチによる探索を試す。

# ランダムサーチ：sklearn.datasets.fetch_california_housing

## パッケージインポート

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense  # layerクラスを直接インポートして使用出来る

##### これうまくいくはずなんだけど学習できてない #####
from tensorflow.keras.losses import MeanSquaredError  # 損失関数クラスを直接インポートして使用出来る
from tensorflow.keras.optimizers import SGD  # オプティマイザクラスを直接インポートして使用出来る
##################################################

import numpy as np
import pandas as pd

print(f'tensorflow ver.{tf.__version__}')
print(f'keras ver.{keras.__version__}')

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tensorflow ver.2.7.0
keras ver.2.7.0


## データロードと前処理

###  データロード（sklearn.datasets.fetch_california_housing）

In [2]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

### validation分割

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

# データサイズを確認
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_valid.shape : {x_valid.shape}')
print(f'y_valid.shape : {y_valid.shape}')

x_train.shape : (11610, 8)
y_train.shape : (11610,)
x_valid.shape : (3870, 8)
y_valid.shape : (3870,)


trainデータを確認

In [4]:
pd_x_train = pd.DataFrame(x_train, columns=housing.feature_names)
display(pd_x_train)
pd_x_train.info()
pd_x_train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.6250,33.0,10.909091,2.227273,63.0,2.863636,40.63,-120.35
1,3.0321,32.0,4.292242,1.045696,2675.0,2.842721,34.23,-118.53
2,4.7222,21.0,5.428571,1.300000,398.0,5.685714,33.92,-117.91
3,2.1111,14.0,5.253763,0.961290,883.0,1.898925,34.50,-117.31
4,4.1667,12.0,6.628571,1.057143,79.0,2.257143,34.25,-119.19
...,...,...,...,...,...,...,...,...
11605,5.7876,33.0,6.117647,0.996324,759.0,2.790441,37.27,-122.00
11606,2.4167,37.0,5.333333,1.202899,564.0,4.086957,35.85,-119.12
11607,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,34.18,-118.38
11608,4.9135,52.0,6.676550,1.037736,993.0,2.676550,34.16,-118.25


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11610 entries, 0 to 11609
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      11610 non-null  float64
 1   HouseAge    11610 non-null  float64
 2   AveRooms    11610 non-null  float64
 3   AveBedrms   11610 non-null  float64
 4   Population  11610 non-null  float64
 5   AveOccup    11610 non-null  float64
 6   Latitude    11610 non-null  float64
 7   Longitude   11610 non-null  float64
dtypes: float64(8)
memory usage: 725.8 KB


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000,11610.000000
mean,3.852212,28.561326,5.429993,1.099821,1434.189061,3.160477,35.636252,-119.568343
std,1.891722,12.556300,2.545286,0.507822,1169.254147,13.797543,2.140908,2.004020
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.750000,32.540000,-124.350000
25%,2.562500,18.000000,4.438356,1.005814,789.000000,2.428453,33.930000,-121.797500
50%,3.517900,29.000000,5.210536,1.049059,1167.500000,2.814371,34.250000,-118.485000
75%,4.706725,37.000000,6.048738,1.100000,1722.750000,3.271133,37.720000,-118.000000
max,15.000100,52.000000,132.533333,34.066667,35682.000000,1243.333333,41.950000,-114.310000


### 前処理

#### スケーリング

### [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
データの標準化を行う。

代表的なメソッドは以下：

|メソッド|説明|
|---|---|
|fit()|標準化するための平均と分散を計算する。|
|trasform()|（事前に計算した平均と分散を使用して）標準化を行う。|
|fit_transform()|平均と分散を計算し、標準化を行う。|

In [5]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)  # x_trainの平均・分散を使用する（のはなぜ？）
x_test = scaler.transform(x_test)  # x_trainの平均・分散を使用する（のはなぜ？）

## モデル作成

複数のモデルを比較しやすくするようにモデルを作成する関数を定義しておく。

この関数では引数で渡した層の数、各層のニューロン数、学習率でSequentialモデル作成→SGDオプティマイザでコンパイルまで行い、モデルを返す。

In [6]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[x_train.shape[1]]):
    # Sequentialモデル生成
    model = keras.models.Sequential()
    # InputLayer追加
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    # hiddenLayer追加
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    # OutputLayer追加
    model.add(keras.layers.Dense(1))

    # optimizer生成
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    # compile
    model.compile(loss='mse', optimizer=optimizer)

    return model

In [7]:
# build_modelを使ってKerasRegressorを作る
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

<ipython-input-7-b4790e7feaeb>:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


KerasRegressorオブジェクトはbuild_modelを使って構築されるモデルに薄いラップをかぶせたものである。
（このラップによって？）scikit-learnの回帰モデルを同じようなユーザインタフェースを使用することが出来る。

# 学習と評価

### コールバック設定

In [8]:
# 早期打ち切りのコールバック関数
# 学習打ち切り時に性能が最高だった時の重みを復元するので最良モデルの保存と復元は不要
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)  # patienceで指定したエポック数学習が進まなかったときに学習を打ち切る

In [9]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

print_valid_train_ration_cb = PrintValTrainRatioCallback()

### TensorBoardを使った可視化

In [10]:
import os

# ログ出力のルートディレクトリ
root_dir = os.path.join(os.curdir, 'my_logs')

# ログディレクトリ名を生成する関数
def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

## 学習

scikit-learnのようにfitメソッドで学習する。</br>
ただし引数はbuild_modelの土台になっているkerasモデルに渡される。
その例として以下ではcallbackを渡している。

In [11]:
# scikit-learnのようにfitメソッドで学習.
keras_reg.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb,print_valid_train_ration_cb,tensorboard_cb])

Epoch 1/100
331/363 [==========================>...] - ETA: 0s - loss: 1.4322
val/train: 0.51
363/363 [==============================] - 1s 2ms/step - loss: 1.3631 - val_loss: 0.6892
Epoch 2/100
327/363 [==========================>...] - ETA: 0s - loss: 0.7267
val/train: 0.88
363/363 [==============================] - 1s 1ms/step - loss: 0.7113 - val_loss: 0.6277
Epoch 3/100
332/363 [==========================>...] - ETA: 0s - loss: 0.6042
val/train: 0.96
363/363 [==============================] - 1s 2ms/step - loss: 0.5996 - val_loss: 0.5734
Epoch 4/100
357/363 [============================>.] - ETA: 0s - loss: 0.5530
val/train: 1.00
363/363 [==============================] - 1s 1ms/step - loss: 0.5508 - val_loss: 0.5499
Epoch 5/100
344/363 [===========================>..] - ETA: 0s - loss: 0.5161
val/train: 1.00
363/363 [==============================] - 1s 1ms/step - loss: 0.5192 - val_loss: 0.5203
Epoch 6/100
327/363 [==========================>...] - ETA: 0s - loss: 0.4935
val/tra

## 学習結果の可視化

In [12]:
%load_ext tensorboard

In [13]:
%tensorboard --logdir ./my_logs

## 学習結果の評価

In [14]:
# scikit-learnのようにscoreメソッドで評価(scoreは高いほぼ良い)
mse_test = keras_reg.score(x_test, y_test)
mse_test

162/162 [==============================] - 0s 988us/step - loss: 0.3764


-0.3764442503452301

# 学習済みモデルを使った予測

In [15]:
# サンプル用にデータサイズを限定
x_new = x_test[:3]

# scikit-learnのようにpredictメソッドで予測
y_pred = keras_reg.predict(x_new)
print(f'predict : {y_pred.reshape(-1)}')
print(f'correct : {y_test[:3]}')

predict : [0.6331352 1.0869802 2.2485156]
correct : [0.875 0.852 2.25 ]


## ランダムサーチによるパラメータ探索

ここまででパラメータ探索をするモデル側の準備が出来たので、実際にランダムサーチを使って探索を行う。

## モデルの保存と復元

In [17]:
from scipy.stats import reciprocal  # 逆分布（対数一様分布）
from sklearn.model_selection import RandomizedSearchCV  # ランダムサーチ

param_distribs = {
    'n_hidden' : [0, 1, 2, 3],
    'n_neurons' : np.arange(1, 100),
    'learning_rate' : reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(estimator=keras_reg  # パラメータ探索を行うモデル
                                    , param_distributions=param_distribs  # パラメータ探索を試すパラメータの分布またはリストの辞書
                                    , n_iter=10  # 試行回数
                                    , cv=3  # 交差検証の分割数
                                    )
rnd_search_cv.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb,print_valid_train_ration_cb,tensorboard_cb])

Epoch 1/100
231/242 [===========================>..] - ETA: 0s - loss: 1.6250
val/train: 0.38
242/242 [==============================] - 1s 2ms/step - loss: 1.5827 - val_loss: 0.6067
Epoch 2/100
220/242 [==========================>...] - ETA: 0s - loss: 0.5520
val/train: 0.98
242/242 [==============================] - 0s 1ms/step - loss: 0.5497 - val_loss: 0.5408
Epoch 3/100
227/242 [===========================>..] - ETA: 0s - loss: 0.5529
val/train: 1.05
242/242 [==============================] - 0s 1ms/step - loss: 0.5481 - val_loss: 0.5729
Epoch 4/100
214/242 [=========================>....] - ETA: 0s - loss: 0.5384
val/train: 1.10
242/242 [==============================] - 0s 1ms/step - loss: 0.5483 - val_loss: 0.6027
Epoch 5/100
238/242 [============================>.] - ETA: 0s - loss: 0.5316
val/train: 0.98
242/242 [==============================] - 0s 1ms/step - loss: 0.5302 - val_loss: 0.5219
Epoch 6/100
214/242 [=========================>....] - ETA: 0s - loss: 0.5221
val/tra

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001FD427E7EE0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FD450500D0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

fit()メソッドに渡した引数はその下のkerasモデルにリレーされている。
また、RandomizedSearchCVは交差検証を使用するためx_valid, y_validは使わない。これらは早期打ち切りのために使用される。

次のように最良のパラメータ、スコア、モデルを取得できる。

In [18]:
# 最良モデルのインデックス
print(rnd_search_cv.best_index_)

# 最良モデルのパラメータ
print(rnd_search_cv.best_params_)

# 最良モデルのスコア
print(rnd_search_cv.best_score_)

# 最良モデル
model = rnd_search_cv.best_estimator_.model
print(model.evaluate(x_test, y_test))

8
{'learning_rate': 0.009515767380476589, 'n_hidden': 3, 'n_neurons': 29}
-0.30612248182296753
162/162 [==============================] - 0s 1ms/step - loss: 0.2959
0.2958737015724182
